In [ ]:
from pykrx import stock
from newspaper import Article
import datetime
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from multiprocessing.dummy import Pool as ThreadPool

import pandas as pd
import re

In [ ]:
# 공공데이터포털 api 주소(Without param)
url_stock = "http://api.seibro.or.kr/openapi/service/StockSvc/getKDRSecnInfo"  
api_service_key_stock = [
    "RXhGWArdgsytKaKf0g%2FWxNuo27wXxg4iChLUs9ePc39VvneddFbQ9v9ZXCDWJkdFbhqCvbw9kdMGy%2F%2Bv3it50A%3D%3D",
    "bqvyeN8k%2B8%2BfRLf7p4CNQsUIEL%2BRb4b2YR08MD10RDv3BxHugq6bR1wFEAo8hTau3XgiLcA7bEBoclnMdyBfNQ%3D%3D",
    "zUgkw3obrruAXAW6kZrJnIpK8UUBIrwXrfroSgoDS7NUlSB%2BDz94OTIkkWeP0V%2BzOz81JVtW84bqh1y0HpzcUg%3D%3D",
    "w9Ra19Zqn3%2BLgg2zHoRiZa8zZPdSCXSgFgrgFGUkaYqqQRD6BVKMsUgiRyJqeEuG1pQ86vSioq03IRarAve7sg%3D%3D",
]  # service api key

# 종목 이름 가져오는 코드

day = (datetime.datetime.now() - datetime.timedelta(days=2)).strftime("%Y%m%d")
def getStockCode():
    """
    market: 상장구분 (11=유가증권, 12=코스닥, 13=K-OTC, 14=코넥스, 50=기타비상장)
    """
    url = f"https://api.odcloud.kr/api/GetStockSecuritiesInfoService/v1/getStockPriceInfo?"
    stock_code = 0
    while True:
        api_decode_key_stock = requests.utils.unquote(
            api_service_key_stock[stock_code], encoding="utf-8"
        )

        params = {
            "serviceKey": api_decode_key_stock,
            "mrktCls": "KOSPI",
            "numOfRows": 1000,
            "beginBasDt":day
        }

        response = requests.get(url, params=params)
        if(response.status_code != 200):
            print(response.status_code)
            stock_code += 1
            continue
        xml = BeautifulSoup(response.text, "lxml")
        items = xml.find("items")
        item_list = []
        for item in items:
            try:
                item_dict = {
                    "stockName": item.find("itmsnm").text.strip(),
                    "stockCode": item.find("srtncd").text.strip(),
                    "marketCap": int(item.find("mrkttotamt").text.strip()),
                    "date": [],
                }
            except AttributeError:
                continue
            item_list.append(item_dict)

        return item_list


In [ ]:
item_list = getStockCode()

In [ ]:
marketCapList = sorted(item_list, key=lambda k:k["marketCap"], reverse=True)

first = []
last = []

print(len(marketCapList))
for i in marketCapList[:50]:
    first.append(i)
    
for i in marketCapList[-20:]:
    last.append(i)

In [ ]:
       
for i in tqdm(first):
    df = stock.get_market_ohlcv("20200101", "20220506", i["stockCode"], adjusted=False)
    cond = df["등락률"] > 10
    for j in df[cond].iloc:
        i["date"].append(j.name.strftime("%Y.%m.%d"))
        
for i in tqdm(last):
    df = stock.get_market_ohlcv("20200101", "20220506", i["stockCode"], adjusted=False)
    cond = df["등락률"] > 20
    for j in df[cond].iloc:
        i["date"].append(j.name.strftime("%Y.%m.%d"))


In [ ]:
#각 크롤링 결과 저장하기 위한 리스트 선언 
title_text=[]
link_text=[]
source_text=[]
date_text=[]
contents_text=[]
result_news = []

#엑셀로 저장하기 위한 변수
RESULT_PATH ='"/Users/bumseok/workspace/capstone-2022-07/model"'  #결과 저장할 경로
now = datetime.datetime.now() #파일이름 현 시간으로 저장하기

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}


#날짜 정제화 함수
def date_cleansing(test):
    try:
        #지난 뉴스
        #머니투데이  10면1단  2018.11.05.  네이버뉴스   보내기  
        pattern = '\d+.(\d+).(\d+).'  #정규표현식 
    
        r = re.compile(pattern)
        match = r.search(test).group(0)  # 2018.11.05.
        date_text.append(match)
        
    except AttributeError:
        #최근 뉴스
        #이데일리  1시간 전  네이버뉴스   보내기  
        pattern = '\w* (\d\w*)'     #정규표현식 
        
        r = re.compile(pattern)
        match = r.search(test).group(1)
        #print(match)
        date_text.append(match)


#내용 정제화 함수 
def contents_cleansing(contents):
    first_cleansing_contents = re.sub('<dl>.*?</a> </div> </dd> <dd>', '', 
                                      str(contents)).strip()  #앞에 필요없는 부분 제거
    second_cleansing_contents = re.sub('<ul class="relation_lst">.*?</dd>', '', 
                                       first_cleansing_contents).strip()#뒤에 필요없는 부분 제거 (새끼 기사)
    third_cleansing_contents = re.sub('<.+?>', '', second_cleansing_contents).strip()
    contents_text.append(third_cleansing_contents)
    #print(contents_text)
    

def crawler(maxpage,query,sort,s_date,e_date):

    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    page = 1  
    maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지
    result = {}
    
    while page <= maxpage_t:
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + query + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        
        response = requests.get(url, headers=headers)
        html = response.text
 
        #뷰티풀소프의 인자값 지정
        soup = BeautifulSoup(html, 'html.parser')
 
        #<a>태그에서 제목과 링크주소 추출
        atags = soup.select('.news_tit')
        for atag in atags:
            link_text.append(atag['href'])   #링크주소
        
        page += 10

In [ ]:
for i in tqdm(first):
    if len(i["date"]) == 0:
        continue
    for j in i["date"]:
        crawler(3, i["stockName"], "0", j, j)
        
for i in tqdm(last):
    if len(i["date"]) == 0:
        continue
    for j in i["date"]:
        crawler(3, i["stockName"], "0", j, j)
        

In [ ]:
news_list = []

def getNews(url):
    article = Article(url, encoding="CP949")
    article.download()
    try:
        article.parse()
        if article.text == "" or article.title == "" or article.publish_date is None or article.url == "":
            return
        news_list.append({"title": article.title, "content": article.text, "time": article.publish_date.strftime("%Y-%m-%d"), "url": article.url})
    except:
        return
    
            
pool = ThreadPool(10)

for _ in tqdm(pool.imap_unordered(getNews, link_text), total=len(link_text)):
    pass

pool.close() 
pool.join()

In [ ]:
link_df = pd.DataFrame(news_list, columns=["title", "content", "date", "url"])
link_df.to_csv("news.csv")